In [1]:
#pip install transformers==4.15.0

In [2]:
!nvidia-smi

Mon Jan 24 20:24:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.94       Driver Version: 470.94       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   30C    P0    18W / 370W |     58MiB / 24265MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
pwd

'/home/apsisdev/mobassir'

In [4]:
ls data

 DataFile/                       memo_yolov5/
 filesharing/                    runs/
 iam_words/                      tensorflow-great-barrier-reef/
 iam_words_merged               'torch seti.ipynb'
 iam_words_merger                Total-Text/
 jigsaw-toxic-severity-rating/   volo/
 Memo_Classifier_pytorch.ipynb   yolo_data/
 memo_cls/                       yolov5/
 memo_resnext50.onnx             yolov5Infer.py
 Memo_yolo_pytorch.ipynb


In [5]:
import transformers
transformers.__version__

'4.15.0'

In [6]:
# %%capture
# !pip install detoxify

In [7]:
# !pip install alt-profanity-check

In [8]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from random import randrange

%matplotlib inline

from detoxify import Detoxify
from transformers import pipeline

from random import randrange
import random

In [9]:
%%time
canonical = 1
text = 1

generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)

data = pd.read_csv('/home/apsisdev/mobassir/data/jigsaw-toxic-severity-rating/validation_data.csv')

model = Detoxify('original')

CPU times: user 15.7 s, sys: 3.66 s, total: 19.4 s
Wall time: 48.9 s


In [10]:
len(data.less_toxic),len(data.more_toxic)

(30108, 30108)

In [11]:
# from profanity_check import predict, predict_prob

# bad = []
# for i in range(len(data.more_toxic)):
#     ck = data.more_toxic[i]
#     bad.append(predict_prob([ck]))
# c = len([i for i in bad if i > 0.5])
# c

In [12]:
profane = pd.read_csv('/home/apsisdev/mobassir/profanity/profanity_en.csv')
profane

,text,canonical_form_1,canonical_form_2,canonical_form_3,category_1,category_2,category_3,severity_rating,severity_description
0,69,69,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
1,@55,ass,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
2,@ssfcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
3,@ssfucker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
4,@ssfvcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.4,Strong
...,...,...,...,...,...,...,...,...,...
1593,wnker,wank,NaN,NaN,sexual anatomy / sexual acts,other / general insult,NaN,1.0,Mild
1594,wop,wop,NaN,NaN,racial / ethnic slurs,NaN,NaN,2.4,Strong
1595,wophead,wop,NaN,NaN,racial / ethnic slurs,NaN,NaN,2.2,Strong
1596,zip in the wire,zipperhead,NaN,NaN,racial / ethnic slurs,NaN,NaN,1.4,Mild


In [ ]:
%%time

less = []
more = []
if(canonical):
    print("....canonical....\n")
    less_count = 0
    more_count = 0
    limit = 10000
    threshold = 1.6
    while(1):

        if(less_count == limit and more_count == limit):
            break
        #ind = randrange(len(profane.severity_rating))
        ind = random.randint(0, len(profane.severity_rating)-1)
        text = generator(profane.canonical_form_1[ind], do_sample=True, min_length=50)
        #text = generator(profane.text[ind], do_sample=True, min_length=50)
        pred = model.predict(text[0]['generated_text'])
        if((profane.severity_rating[ind]<threshold) and pred['toxicity'] < 0.5 and less_count<limit):
            less.append(text[0]['generated_text'])
            less_count+=1
        elif((profane.severity_rating[ind]>=threshold) and pred['toxicity'] >= 0.5 and more_count<limit):
            more.append(text[0]['generated_text'])
            more_count+=1
        #print('less_count : ',less_count)
        #print('more_count : ',more_count)

if(text):    
    print("....full text....\n\n\n\n\n\n")
    less_count = 0
    more_count = 0
    limit = 10000
    threshold = 1.6
    while(1):

        if(less_count == limit and more_count == limit):
            break
        #ind = randrange(len(profane.severity_rating))
        ind = random.randint(0, len(profane.severity_rating)-1)
        #text = generator(profane.canonical_form_1[ind], do_sample=True, min_length=50)
        text = generator(profane.text[ind], do_sample=True, min_length=60)
        pred = model.predict(text[0]['generated_text'])
        if((profane.severity_rating[ind]<threshold) and pred['toxicity'] < 0.5 and less_count<limit):
            less.append(text[0]['generated_text'])
            less_count+=1
        elif((profane.severity_rating[ind]>=threshold) and pred['toxicity'] >= 0.5 and more_count<limit):
            more.append(text[0]['generated_text'])
            more_count+=1
        #print('less_count : ',less_count)
        #print('more_count : ',more_count)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


....canonical....



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/apsisdev/anaconda3/envs/mobassir/lib/python3.8/site-packages/transformers/pipelines/base.py:1075: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
df = pd.DataFrame(columns=['less_toxic','more_toxic'])
df

In [ ]:
df['less_toxic'] = less
df['more_toxic'] = more
df

In [ ]:
df.to_csv('/home/apsisdev/mobassir/data/jigsaw-toxic-severity-rating/faketoxicity.csv',index = False)

In [ ]:
len(more)

In [ ]:
!nvidia-smi

In [ ]:
#len(profane.severity_rating)